<a href="https://colab.research.google.com/github/taba4ek555/dz/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Доработать блокнот

### Задача 1. Стиль-трансфер диалогов в разные эпохи
#### Постановка
Необходимо преобразовать современный диалог в стиле «XXI века» к диалогу другой эпохи (например, рубежа XIX–XX вв.) или жанра (фэнтези, научная фантастика и т.д.), сохранив при этом общий смысл исходной беседы.
#### Стартовый датасет
    • DailyDialog — содержит повседневные короткие диалоги. Его можно дополнить собственным корпусом литературы или сценариев разных эпох (например, тексты из Project Gutenberg).
#### Метрики
    1. BLEU / ROUGE / BERTScore: Оценка схожести с «целью» стилевого трансфера (при наличии эталонных примеров).
    2. Content Preservation: Специальные метрики (часто основанные на similar sentence embeddings), чтобы проверить, сохраняется ли исходный смысл.
    3. Style Accuracy: Классификатор стиля (обученный отдельно на текстах целевой эпохи/жанра) оценивает, насколько успешно изменён стиль.


In [ ]:
# =============================================
# Пример кода для Задания 2: "Стиль-трансфер диалогов"
# =============================================
# -------------------------
# Шаг 0. Установка библиотек
# -------------------------
# В средах вроде Google Colab можно раскомментировать команду !pip install ...
# В Jupyter на локальном компьютере установите необходимые пакеты вручную.

!pip install transformers datasets sentencepiece
!pip install rouge-score
!pip install sacrebleu
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2f5734e1b2ca58dfa74595d3c179c9f704b3fb44eac1ca32dc2a2056dbd1c70f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from rouge_score import rouge_scorer
import sacrebleu
from bert_score import score


In [ ]:

# =============================================
# Шаг 1. Подготовка данных
# =============================================
# Представим, что у нас есть небольшой набор "современных" диалогов,
# и мы хотим перевести их в стиль "старой/архаичной" английской речи (условный пример).

data = [
    {
        "modern_dialog": "Hey, how are you doing today? Everything going well?",
        # "reference_style" - эталонная фраза в нужном стиле (для метрик)
        "reference_style": "Greetings, how dost thou fare this day? All proceedeth well?"
    },
    {
        "modern_dialog": "Hello sir, can you tell me where the library is?",
        "reference_style": "Good morrow, couldst thou direct me to the grand library?"
    }
]

In [ ]:

# =============================================
# Шаг 2. Загрузка модели для стиль-трансфера
# =============================================
# Для демонстрации используем модель T5 (англоязычную).
# В реальном проекте потребуется дообучение (fine-tuning) на корпусе "старинной" речи.
# Здесь просто показываем принцип: подача на вход промпта с задачей "transfer to old english style: ..."

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

# =============================================
# Шаг 3. Генерация преобразованных диалогов
# =============================================
generated_texts = []

for item in data:
    prompt = f"translate to old english style: {item['modern_dialog']}"
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Генерация
    outputs = model.generate(
        inputs,
        max_length=50,
        num_beams=4,
        early_stopping=True
    )

    styled_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    generated_texts.append({
        "modern_dialog": item["modern_dialog"],
        "styled_text": styled_text,
        "reference_style": item["reference_style"]
    })

In [ ]:

# =============================================
# Шаг 4. Вывод результатов
# =============================================
print("=== Результаты стиль-трансфера ===\n")
for idx, res in enumerate(generated_texts):
    print(f"Modern dialog:      {res['modern_dialog']}")
    print(f"Generated (styled): {res['styled_text']}")
    print(f"Reference style:    {res['reference_style']}")
    print("-" * 80)

=== Результаты стиль-трансфера ===

Modern dialog:      Hey, how are you doing today? Everything going well?
Generated (styled): Hey, how are you doing today?
Reference style:    Greetings, how dost thou fare this day? All proceedeth well?
--------------------------------------------------------------------------------
Modern dialog:      Hello sir, can you tell me where the library is?
Generated (styled): Hello sir, can you tell me where the library is?
Reference style:    Good morrow, couldst thou direct me to the grand library?
--------------------------------------------------------------------------------


In [ ]:
generated_texts[0]

{'modern_dialog': 'Hey, how are you doing today? Everything going well?',
 'styled_text': 'Hey, how are you doing today?',
 'reference_style': 'Greetings, how dost thou fare this day? All proceedeth well?'}

BLEU основаны на подсчете совпадений слов и словосочетаний в машинном переводе и в эталоне. К итоговому precision применяется корректировка — штраф за краткость (brevity penalty), чтобы избежать слишком высоких оценок.

Для ROUGE считается не только precision, но и recall, а также параметр F1. Это позволяет обойтись без штрафа за краткость(чаще всего используется для оценки текстовых резюме)
```
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3'], use_stemmer=True)
```



BERT вычисляет косинусное сходство между контекстуальными выстраиваниями слов в эталонном тексте и сгенерированном тексте. Эти сходства затем агрегируются для получения итогового балла. Чем выше BERTScore, тем больше семантическое совпадение между выводом LLM и эталонным текстом. (уязвимы к контекстуальной осведомленности и предвзятости из-за их зависимости от контекстуальных встраиваний, полученных из предварительно обученных моделей)


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [ ]:

# =============================================
# Шаг 5. Оценка по метрикам (BLEU, ROUGE, BERTScore)
# =============================================

from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score_fn

# Подготовка данных для оценки
references = [item["reference_style"] for item in generated_texts]
hypotheses = [item["styled_text"] for item in generated_texts]

# BLEU
bleu_score = corpus_bleu(hypotheses, [references])
print(f"BLEU Score: {bleu_score.score}")

# ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3'], use_stemmer=True)
rouge_scores = {
    "rouge1": [],
    "rouge2": [],
    "rouge3": []
}

for ref, hyp in zip(references, hypotheses):
    scores = scorer.score(ref, hyp)
    for metric, value in scores.items():
        rouge_scores[metric].append(value.fmeasure)

# Средние значения для каждой метрики ROUGE
avg_rouge_scores = {metric: sum(values) / len(values) for metric, values in rouge_scores.items()}
print(f"ROUGE Scores: {avg_rouge_scores}")

# BERTScore
P, R, F1 = bert_score_fn(hypotheses, references, lang="en", verbose=True)
print(f"BERTScore (Precision): {P.mean().item()}")
print(f"BERTScore (Recall): {R.mean().item()}")
print(f"BERTScore (F1): {F1.mean().item()}")



BLEU Score: 4.621561639385843
ROUGE Scores: {'rouge1': 0.2125, 'rouge2': 0.0, 'rouge3': 0.0}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.78 seconds, 2.56 sentences/sec
BERTScore (Precision): 0.8951449394226074
BERTScore (Recall): 0.8454194068908691
BERTScore (F1): 0.8695480823516846


In [ ]:

# =============================================
# Шаг 6. Content Preservation
# =============================================
# Для упрощённой оценки сохранившегося смысла сравним с "исходным" текстом.
# Снова используем BERTScore, но теперь "reference" = modern_dialog.

# Подготовка данных для оценки
references = [item["modern_dialog"] for item in generated_texts]
hypotheses = [item["styled_text"] for item in generated_texts]

# BERTScore
P, R, F1 = bert_score_fn(hypotheses, references, lang="en", verbose=True)
print(f"BERTScore (Precision): {P.mean().item()}")
print(f"BERTScore (Recall): {R.mean().item()}")
print(f"BERTScore (F1): {F1.mean().item()}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.13 seconds, 15.18 sentences/sec
BERTScore (Precision): 0.9948806762695312
BERTScore (Recall): 0.97374427318573
BERTScore (F1): 0.9840818643569946


In [ ]:

# =============================================
# Шаг 7. Style Accuracy
# =============================================
# Для строгой оценки "насколько текст выглядит стилизованным под староанглийский"
# обычно тренируют или используют готовый классификатор стилей (модель для text-classification).
# Здесь сделаем упрощённый "заглушечный" пример.

# Допустим, у нас есть условный простейший классификатор, который считает, что
# если текст содержит "thou", "thee", "dost" и т.д., это "old english", иначе "modern".
# В реальном проекте нужно собрать датасет и обучить/использовать модель.

def dummy_style_classifier(text: str) -> str:
    # Если находит хотя бы одно "староанглийское" слово, предполагаем класс "old_english", иначе "modern"
    old_words = {"thou", "thee", "thy", "thine", "dost"}
    text_lower = text.lower()
    if any(keyword in text_lower for keyword in old_words):
        return "old_english"
    else:
        return "modern"

style_predictions = [dummy_style_classifier(item["styled_text"]) for item in generated_texts]
references_style = [item["reference_style"] for item in generated_texts]
# Подсчет точности
correct_predictions = sum(1 for pred, ref in zip(style_predictions, references_style) if pred == ref)
total_samples = len(generated_texts)
accuracy = correct_predictions / total_samples if total_samples > 0 else 0
print(f"Style Accuracy: {accuracy:.2f}")

Style Accuracy: 0.00


In [ ]:

# =============================================
# Шаг 8. Итоги
# =============================================
print("\n=== Вывод ===")
print("В примере выше показана базовая демонстрация использования T5 для 'стиль-трансфера'.")
print("В реальных задачах рекомендуется:")
print("  - Собрать и разметить датасет примеров старинной речи и её соответствий.")
print("  - Выполнить дообучение модели (fine-tuning).")
print("  - Использовать полноценный классификатор стиля, обученный на стилизованных текстах.")
print("  - Проводить ручную оценку (human evaluation) для проверки качества переноса стиля.")



=== Вывод ===
В примере выше показана базовая демонстрация использования T5 для 'стиль-трансфера'.
В реальных задачах рекомендуется:
  - Собрать и разметить датасет примеров старинной речи и её соответствий.
  - Выполнить дообучение модели (fine-tuning).
  - Использовать полноценный классификатор стиля, обученный на стилизованных текстах.
  - Проводить ручную оценку (human evaluation) для проверки качества переноса стиля.
